In [1]:
import requests
import lxml.html as lh
import pandas as pd

## Récupération des données par web Scraping

Récupération d'une liste de jeu simple avec nom, desription et date de sortie : http://www.jeuxvideo.com/meilleurs/machine-20/

In [15]:
url='http://www.jeuxvideo.com/meilleurs/'
page = requests.get(url)
doc = lh.fromstring(page.content)

On récupère les données via des requêtes XPath

In [29]:
nomJeux = doc.xpath('//a[@class="gameTitleLink__196nPy"]')
desc = doc.xpath('//p[@class="description__1-Pqha"]')
sortie = doc.xpath('//span[@class="releaseDate__1RvUmc"]')
test = doc.xpath('//span[@class="editorialRating__1tYu_r"]')
avis = doc.xpath('//span[@class="userRating__1y96su"]')

Affichage des données décomposées :

In [34]:
for i in range(len(nomJeux)):
    print(nomJeux[i].text_content().strip() + "\n" + \
          desc[i].text_content().strip() + "\n" + \
          sortie[i].text_content().strip()+ "\n" + \
          test[i].text_content().strip() + "\n" + \
          avis[i].text_content().strip() + "\n")

God of War sur PS4
Dans ce nouvel épisode de God Of War, le héros évoluera dans un monde aux inspirations nordiques, très forestier et montagneux. Dans ce beat-them-all, un enfant accompagnera le héros principal, pouvant apprendre des actions du joueur, et même gagner de l'expérience.
Sortie: 20 avril 2018
20/20
17.7/20

The Legend of Zelda : Breath of the Wild
The Legend of Zelda : Breath of the Wild est un jeu d'action/aventure. Link se réveille d'un sommeil de 100 ans dans un royaume d'Hyrule dévasté. Il lui faudra percer les mystères du passé et vaincre Ganon, le fléau. L'aventure prend place dans un gigantesque monde ouvert et accorde ainsi une part importante à l'exploration. Le titre a été pensé pour que le joueur puisse aller où il veut dès le début, s'éloignant de la linéarité habituelle de la série.
Sortie: 3 mars 2017
20/20
17.5/20

The Legend of Zelda : The Wind Waker sur NGC
Dans The Legend of Zelda : The Wind Waker, vous incarnez Link dans sa quête pour sauver sa sœur. Dé

IndexError: list index out of range

### Mettons les données dans un DataFrame Pandas

In [37]:
l1 = pd.DataFrame([nomJeux[i].text_content().strip() for i in range(len(nomJeux))], columns=['Jeux'])
l1['key'] = l1.index
l2 = pd.DataFrame([desc[i].text_content().strip() for i in range(len(desc))], columns=['Description'])
l2['key'] = l2.index
l3 = pd.DataFrame([sortie[i].text_content().strip().replace("Sortie: ", "") for i in range(len(sortie))], columns=['Sortie'])
l3['key'] = l3.index
l4 = pd.DataFrame([test[i].text_content().strip() for i in range(len(test))], columns=['Test'])
l4['key'] = l4.index
l5 = pd.DataFrame([avis[i].text_content().strip() for i in range(len(avis))], columns=['Avis'])
l5['key'] = l5.index

liste = l1.join(l2, on='key', how='left', lsuffix='_l1', rsuffix='_r1').join(l3, on='key_l1', how='left', lsuffix='_l1', rsuffix='_r2')
del liste['key_l1']
del liste['key_r1']
liste = liste.join(l4, on='key', how='left', lsuffix='_l1', rsuffix='_r1').join(l5, on='key_l1', how='left', lsuffix='_l1', rsuffix='_r2')
del liste['key_l1']
del liste['key_r1']
del liste['key']
liste

,Jeux,Description,Sortie,Test,Avis
0,God of War sur PS4,"Dans ce nouvel épisode de God Of War, le héros...",20 avril 2018,20/20,17.7/20
1,The Legend of Zelda : Breath of the Wild,The Legend of Zelda : Breath of the Wild est u...,3 mars 2017,20/20,17.5/20
2,The Legend of Zelda : The Wind Waker sur NGC,"Dans The Legend of Zelda : The Wind Waker, vou...",3 mai 2003,20/20,18.9/20
3,Silent Hill,Silent Hill est un survival horror à télécharg...,1er août 1999,19/20,18.6/20
4,Panzer Dragoon Saga sur Saturn,Panzer Dragoon Saga est un jeu de rôle en 3D s...,Juin 1998,19/20,18.8/20
5,Kingpin : Life of Crime sur PC,Kingpin : Life of Crime est un jeu de tir à la...,Août 1999,19/20,17.5/20
6,ISS Pro Evolution sur PS1,ISS Pro Evolution est un jeu de football sur P...,25 février 2000,19/20,18.4/20
7,Batman Arkham City,Batman Arkham City est un jeu d'aventure/actio...,20 octobre 2011,19/20,18.6/20
8,Super Metroid,Retrouvez la chasseuse de primes intergalactiq...,28 juillet 1994,19/20,19.4/20
9,Umineko no Naku Koro ni : Shinjitsu to Gensô n...,Umineko no Naku Koro ni est un visual novel. L...,8 juillet 2016,19/20,18.6/20


## Création de la fonction

In [58]:
def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    nomJeux = doc.xpath('//a[@class="gameTitleLink__196nPy"]')
    desc = doc.xpath('//p[@class="description__1-Pqha"]')
    sortie = doc.xpath('//span[@class="releaseDate__1RvUmc"]')
    test = doc.xpath('//span[@class="editorialRating__1tYu_r"]')
    avis = doc.xpath('//span[@class="userRating__1y96su"]')
    
    l1 = pd.DataFrame([nomJeux[i].text_content().strip() for i in range(len(nomJeux))], columns=['Jeux'])
    l1['key'] = l1.index
    l2 = pd.DataFrame([desc[i].text_content().strip() for i in range(len(desc))], columns=['Description'])
    l2['key'] = l2.index
    l3 = pd.DataFrame([sortie[i].text_content().strip().replace("Sortie: ", "") for i in range(len(sortie))], columns=['Sortie'])
    l3['key'] = l3.index
    l4 = pd.DataFrame([test[i].text_content().strip() for i in range(len(test))], columns=['Test'])
    l4['key'] = l4.index
    l5 = pd.DataFrame([avis[i].text_content().strip() for i in range(len(avis))], columns=['Avis'])
    l5['key'] = l5.index
    
    liste = pd.DataFrame()
    liste = l1.join(l2, on='key', how='left', lsuffix='_l1', rsuffix='_r1').join(l3, on='key_l1', how='left', lsuffix='_l1', rsuffix='_r2')
    del liste['key_l1']
    del liste['key_r1']
    liste = liste.join(l4, on='key', how='left', lsuffix='_l1', rsuffix='_r1').join(l5, on='key_l1', how='left', lsuffix='_l1', rsuffix='_r2')
    del liste['key_l1']
    del liste['key_r1']
    del liste['key']
    
    return liste

getPage('http://www.jeuxvideo.com/meilleurs/')

,Jeux,Description,Sortie,Test,Avis
0,God of War sur PS4,"Dans ce nouvel épisode de God Of War, le héros...",20 avril 2018,20/20,17.7/20
1,The Legend of Zelda : Breath of the Wild,The Legend of Zelda : Breath of the Wild est u...,3 mars 2017,20/20,17.5/20
2,The Legend of Zelda : The Wind Waker sur NGC,"Dans The Legend of Zelda : The Wind Waker, vou...",3 mai 2003,20/20,18.9/20
3,Silent Hill,Silent Hill est un survival horror à télécharg...,1er août 1999,19/20,18.6/20
4,Panzer Dragoon Saga sur Saturn,Panzer Dragoon Saga est un jeu de rôle en 3D s...,Juin 1998,19/20,18.8/20
5,Kingpin : Life of Crime sur PC,Kingpin : Life of Crime est un jeu de tir à la...,Août 1999,19/20,17.5/20
6,ISS Pro Evolution sur PS1,ISS Pro Evolution est un jeu de football sur P...,25 février 2000,19/20,18.4/20
7,Batman Arkham City,Batman Arkham City est un jeu d'aventure/actio...,20 octobre 2011,19/20,18.6/20
8,Super Metroid,Retrouvez la chasseuse de primes intergalactiq...,28 juillet 1994,19/20,19.4/20
9,Umineko no Naku Koro ni : Shinjitsu to Gensô n...,Umineko no Naku Koro ni est un visual novel. L...,8 juillet 2016,19/20,18.6/20


## Gestion de la pagination maintenant

In [75]:
nbPages = 10
def getPages(_nbPages, _url):
    liste_finale = pd.DataFrame()
    for i in range (_nbPages):
        liste = getPage(_url + '?p=' + str(i+1))
        liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
    return liste_finale

liste_totale = getPages(nbPages, 'http://www.jeuxvideo.com/meilleurs/')
liste_totale

,Jeux,Description,Sortie,Test,Avis
0,God of War sur PS4,"Dans ce nouvel épisode de God Of War, le héros...",20 avril 2018,20/20,17.7/20
1,The Legend of Zelda : Breath of the Wild,The Legend of Zelda : Breath of the Wild est u...,3 mars 2017,20/20,17.5/20
2,The Legend of Zelda : The Wind Waker sur NGC,"Dans The Legend of Zelda : The Wind Waker, vou...",3 mai 2003,20/20,18.9/20
3,Silent Hill,Silent Hill est un survival horror à télécharg...,1er août 1999,19/20,18.6/20
4,Panzer Dragoon Saga sur Saturn,Panzer Dragoon Saga est un jeu de rôle en 3D s...,Juin 1998,19/20,18.8/20
5,Kingpin : Life of Crime sur PC,Kingpin : Life of Crime est un jeu de tir à la...,Août 1999,19/20,17.5/20
6,ISS Pro Evolution sur PS1,ISS Pro Evolution est un jeu de football sur P...,25 février 2000,19/20,18.4/20
7,Batman Arkham City,Batman Arkham City est un jeu d'aventure/actio...,20 octobre 2011,19/20,18.6/20
8,Super Metroid,Retrouvez la chasseuse de primes intergalactiq...,28 juillet 1994,19/20,19.4/20
9,Umineko no Naku Koro ni : Shinjitsu to Gensô n...,Umineko no Naku Koro ni est un visual novel. L...,8 juillet 2016,19/20,18.6/20


Ecrit la liste dans un fichier

In [76]:
liste_totale.to_csv('meilleursjeuvideo.csv', index=False)